In [2]:
from azureml.core import Workspace, Experiment, Environment
from azureml.core.authentication import InteractiveLoginAuthentication

workspace = "quick-starts-ws-165228"
resource_name = "aml-quickstarts-165228"
# subscription_id = "9a7511b8-150f-4a58-8528-3e7d50216c31"
subscription_id = "81cefad3-d2c9-4f77-a466-99a7f541c7bb"
ws = Workspace.get(subscription_id=subscription_id,
                   resource_group=resource_name,
                   name=workspace)
exp = Experiment(workspace=ws, name="udacity-project-1")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()
Environment.list(workspace=ws)

Workspace name: quick-starts-ws-165228
Azure region: southcentralus
Subscription id: 81cefad3-d2c9-4f77-a466-99a7f541c7bb
Resource group: aml-quickstarts-165228


{'AzureML-Triton': {
     "databricks": {
         "eggLibraries": [],
         "jarLibraries": [],
         "mavenLibraries": [],
         "pypiLibraries": [],
         "rcranLibraries": []
     },
     "docker": {
         "arguments": [],
         "baseDockerfile": null,
         "baseImage": "mcr.microsoft.com/azureml/aml-triton:20210811.v1",
         "baseImageRegistry": {
             "address": null,
             "password": null,
             "registryIdentity": null,
             "username": null
         },
         "enabled": false,
         "platform": {
             "architecture": "amd64",
             "os": "Linux"
         },
         "sharedVolumes": true,
         "shmSize": null
     },
     "environmentVariables": {
         "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
     },
     "inferencingStackVersion": null,
     "name": "AzureML-Triton",
     "python": {
         "baseCondaEnvironment": null,
         "condaDependencies": {
             "channels": [
                 "

In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "cpucluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # To use a different region for the compute, add a location='<region>' parameter
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [20]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os
from azureml.core import ScriptRunConfig

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        '--C': uniform(0.1, 2.0),
        '--max_iter': choice(10,50,100)
    }
)

# Specify a Policy
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=3)



if "training" not in os.listdir():
    os.mkdir("./training")
    
#env = Environment.from_conda_specification(name='sklearn-env', file_path='./conda_dep.yml')

#env = Environment.from_pip_requirements('sklearn-env', 'requirements.txt')

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory='.',
                             entry_script='train.py',
                             compute_target=cpu_cluster,
                             conda_packages=['py-xgboost','pandas'])

# Create a SKLearn estimator for use with train.py
# config = ScriptRunConfig(source_directory='.',
#                             script='train.py',
#                             compute_target=cpu_cluster,
#                             environment=env)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name="Accuracy",
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=20,
                                     max_concurrent_runs=4)

In [21]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hdr = exp.submit(config = hyperdrive_config)
RunDetails(hdr).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [23]:
import joblib
# Get your best run and save the model from that run.

best_run = hdr.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']



In [24]:
best_run_metrics

{'Regularization Strength:': 0.6070316496944049,
 'Max iterations:': 100,
 'Accuracy': 0.91350531107739}

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(### YOUR DATA OBJECT HERE ###)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task=,
    primary_metric=,
    training_data=,
    label_column_name=,
    n_cross_validations=)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###